In [23]:
import pandas as pd
import json

In [18]:
df = pd.read_csv("./expo_chats.csv",encoding="utf-8")

In [19]:
print(df.head)
print(df.columns)

<bound method NDFrame.head of            sender_name   timestamp_ms   
0            Expo Shop  1591635497731  \
1            Expo Shop  1591635498753   
2     Ayman K Abdallah  1591635499161   
3            Expo Shop  1591635499761   
4     Ayman K Abdallah  1591635501168   
...                ...            ...   
1746         Expo Shop  1596560467595   
1747         Expo Shop  1595750297089   
1748         Expo Shop  1597512378235   
1749         Expo Shop  1595276814403   
1750         Expo Shop  1602259545256   

                                            sticker.uri   
0                                                   NaN  \
1                                                   NaN   
2                                                   NaN   
3                                                   NaN   
4                                                   NaN   
...                                                 ...   
1746                                                NaN   
1747 

In [20]:
df = df.drop('sticker.uri', axis=1)
df = df.iloc[:, :3]
print(df.columns)

Index(['sender_name', 'timestamp_ms', 'content'], dtype='object')


In [21]:
print(df['content'].isna().any())
df['content'].fillna('<emoji>', inplace=True)
print(df['content'].isna().any())

True
False


In [22]:
df = df.sort_values('timestamp_ms')
print(df.head)

<bound method NDFrame.head of              sender_name   timestamp_ms   
466            Expo Shop  1582973415003  \
489            Expo Shop  1582973415877   
522            عنان مراد  1582973416097   
556            Expo Shop  1582973416589   
609            عنان مراد  1582973432081   
...                  ...            ...   
1464        Nuhad Joudeh  1673358658138   
1622           Expo Shop  1673358658989   
1316  Mohannad Abed Rabu  1681173954698   
1466           Expo Shop  1681173956142   
1587  Mohannad Abed Rabu  1681173961791   

                                                content  
466               تم الرد على إعلان بواسطة ‏عنان مراد‏.  
489                       مرحبا عنان\nكيف بنقدر نساعدك؟  
522                        هل المنتجات المعروضة مكفولة؟  
556   جميع المنتجات المعروضة لدينا مكفولة لمدة سنة ك...  
609                                                   👍  
...                                                 ...  
1464                    تم إرسال مرفق بواسطة ‏‎

In [77]:
def extract_name(sentence):
    prefix = "بواسطة ‏"
    suffix = "‏."

    start_index = sentence.find(prefix) + len(prefix)
    end_index = sentence.find(suffix)

    if start_index != -1 and end_index != -1:
        return sentence[start_index:end_index]
    else:
        return None

def clean_content(msg,usr):
    if "تم الرد على إعلان" in msg:
        usr = extract_name(msg)
        msg = "مرحبا, عندي استفسار عن هذا المنتج <product>" 
    if "Please let us know how we can help you." in msg or "نحن الآن خارج وقت العمل الرسمي و سيتم الرد عليكم في أقرب وقت ممكن" in msg:
        msg = ""
    return msg,usr
def save_json(data,path,shop_name):
    # Convert dictionary to JSON
    output = []
    for chat in data:
        if len(data[chat]) == 0:
            continue
        tmp ={
            "dialog_id": chat,
             "dialog": []
             }
        count = 0
        for dialog in data[chat]:
            
            key, value = list(dialog.items())[0]
            if key == shop_name:
                key = "participant1"
            else:
                key = "participant2"

            tmp["dialog"].append({
                "id": count,
                "sender": key,
                "text": value
            }),

            
            count +=1
        output.append(tmp)

    json_object = json.dumps(output, ensure_ascii=False, indent=4)

    # Save JSON data to a file
    with open(path, "w",encoding="utf-8") as json_file:
        json_file.write(json_object)

    print("JSON data saved to data.json")

In [75]:
cache = dict()
dialogs = dict()
active_dialog = ""
shop_name = "Expo Shop"
for index, row in df.iterrows():
    content,usr = clean_content(row[2],row[0])
    if content == "":
        continue
    if usr != shop_name and usr not in dialogs:
        dialogs[usr] = []
        if active_dialog =="":
            active_dialog = usr
    elif usr != active_dialog and usr != shop_name and shop_name in cache:
        tmp={shop_name:cache[shop_name]}
        dialogs[active_dialog].append(tmp)
        cache.pop(shop_name)
        active_dialog = usr
    elif usr != active_dialog and usr != shop_name and shop_name not in cache:
        tmp={active_dialog:cache[active_dialog]}
        dialogs[active_dialog].append(tmp)
        cache.pop(active_dialog)
        active_dialog = usr
    elif usr == shop_name and active_dialog in cache:
        tmp={active_dialog:cache[active_dialog]}
        dialogs[active_dialog].append(tmp)
        cache.pop(active_dialog)
    elif usr != shop_name and active_dialog == usr and shop_name in cache:
        tmp={shop_name:cache[shop_name]}
        dialogs[active_dialog].append(tmp)
        cache.pop(shop_name)
    if usr in cache:
        if content not in cache[usr]:
            cache[usr]+=","+content
    if usr not in cache:
        cache[usr] = content
    

In [78]:
save_json(dialogs,"./arabic_conversation.json",shop_name)

JSON data saved to data.json


In [72]:
print(dialogs["عنان مراد"])

[{'عنان مراد': 'مرحبا, عندي استفسار'}, {'Expo Shop': 'مرحبا عنان\nكيف بنقدر نساعدك؟'}, {'عنان مراد': 'هل المنتجات المعروضة مكفولة؟'}, {'Expo Shop': 'جميع المنتجات المعروضة لدينا مكفولة لمدة سنة كاملة ( لا تشمل سوء الاستخدام و الكسر)'}, {'عنان مراد': '👍'}, {'Expo Shop': '<emoji>'}, {'عنان مراد': 'كم تبلغ مدة التوصيل؟,كم تبلغ مدة التوصيل؟,كم تبلغ مدة التوصيل؟'}, {'Expo Shop': 'مدة التوصيل من 3 - 5 ايام عمل,مرحبا Sameer\nكيف بنقدر نساعدك؟,شكرا لتواصلك معنا,بنعتذر نفذت الكمية,التوصيل مجاني داخل الضفة الغربية و القدس لجميع الطلبات باستثناء الملابس.,جميع المنتجات المعروضة مكفولة لمدة سنة كاملة لا تشمل سوء الاستخدام'}]
